In [ ]:
import keras_tuner as kt, tensorflow as tf
from src.data_utils import make_generators
from src.config import IMG_SIZE, N_CLASSES, MODELS, HISTORIES
from tensorflow.keras import layers

# 🔧 build function for tuner
def model_builder(hp):
    m = tf.keras.Sequential()
    m.add(layers.Input(shape=(*IMG_SIZE,3)))
    for i in range(hp.Int("conv_blocks", 2, 4)):
        filters = hp.Choice(f"filters_{i}", [32, 64, 128])
        m.add(layers.Conv2D(filters, (3,3), activation='relu', padding='same'))
        m.add(layers.MaxPooling2D())
    m.add(layers.Flatten())
    m.add(layers.Dense(hp.Int("dense_units", 64, 256, step=64),
                       activation='relu'))
    m.add(layers.Dropout(hp.Float("dropout", 0.2, 0.5, step=0.1)))
    m.add(layers.Dense(N_CLASSES, activation='softmax'))
    m.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float("lr", 1e-4, 1e-2, sampling="log")),
        loss="categorical_crossentropy",
        metrics=["accuracy"])
    return m

train_gen, val_gen, _ = make_generators()
tuner = kt.RandomSearch(model_builder,
                        objective="val_accuracy",
                        max_trials=20,
                        directory="kt_logs", project_name="intel")
tuner.search(train_gen, epochs=15, validation_data=val_gen)
best_model = tuner.get_best_models(1)[0]
best_model.save(MODELS/"hp.h5")

# save history
import json, numpy as np
hist = tuner.oracle.get_best_trials(1)[0].metrics.get_history("val_accuracy")
json.dump({"val_accuracy": hist}, open(HISTORIES/"hp.json", "w"))
